## Example of Optimization

### This code snippets provide the examples in Section 5.4.2 of the SLAM Handbook.

Consider a pose estimation problem, where the goal is to find the robot's pose ${y}=\{{R}, {t}\}$ that minimizes the error between the observed measurements of landmarks and the predicted measurements based on the current robot pose. This can be formulated as an optimization problem with a cost function:
    \begin{equation}
        \min_{\theta} \sum_i \left(R {p}_i + {t} - {q}_i\right)^T  \left({R} {p}_i + {t} - {q}_i\right),
    \end{equation}
where ${R} \in SO(3)$ represents the rotation of the robot's body and ${t} \in \mathbb{R}^3$ represents the translation. As illustrated below, the term ${p}_i$ is the position of the $i$-th landmark in the robot's local frame, and ${q}_i$ is the observed position of the $i$-th landmark.

![title](figs/diff-opt.jpg)

In [1]:
from torch import nn
import torch, pypose as pp
from pypose.optim import LM
from pypose.optim.strategy import Constant
from pypose.optim.scheduler import StopOnPlateau

### Model preparation

Define the regression model ${h}({ y}, {p}_i) = {R} {p}_i + {t}$ that computes the predicted landmarks based on the robot's pose.

In [2]:
class InvNet(nn.Module):
    def __init__(self, *dim):
        super().__init__()
        init = pp.randn_SE3(*dim)
        self.pose = pp.Parameter(init)

    def forward(self, input):
        error = (self.pose @ input).Log()
        return error.tensor() # Returns Tensor as Error

### Data preparation

Initialize the parameters with an initial guess for the pose parameters ${ y} = \{{R_0}, {t_0}\}$. 

In [3]:
device = torch.device("cpu") # can also be cuda
input = pp.randn_SE3(2, 2, device=device)
invnet = InvNet(2, 2).to(device)

### Optimizer preparation

Set up the LM optimizer, including robust kernel functions and correctors to handle outliers. 

In [4]:
strategy = Constant(damping=1e-4)
optimizer = LM(invnet, strategy=strategy)
scheduler = StopOnPlateau(optimizer, steps=10, patience=3, decreasing=1e-3, verbose=True)

### Optimization

Perform the optimization by iteratively updating the parameters. 
The following sample code illustrates the use of PyPose to solve this problem by providing two options for calling the optimizer, assuming the target pose is the identity pose for simplicity.

The first one is full optimization, suitable for application.

The second one is step optimization, suitable for visulizing intermedium output.

Users only need to select one of them in application.

In [5]:
# 1st option, full optimization
scheduler.optimize(input=input)

StopOnPlateau on step 0 Loss 2.333283e+01 --> Loss 9.887070e-07 (reduction/loss: 1.0000e+00).
StopOnPlateau on step 1 Loss 9.887070e-07 --> Loss 3.617341e-14 (reduction/loss: 1.0000e+00).
StopOnPlateau on step 2 Loss 3.617341e-14 --> Loss 1.781908e-14 (reduction/loss: 5.0740e-01).
StopOnPlateau on step 3 Loss 1.781908e-14 --> Loss 5.551115e-17 (reduction/loss: 9.9688e-01).
StopOnPlateau: Maximum patience steps reached, Quiting..


In [6]:
# 2nd option, step optimization
while scheduler.continual():
    loss = optimizer.step(input)
    scheduler.step(loss)